In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy import stats
import datetime as dt
import csv

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [5]:
def getData():
  data = []
  dateNums = []
  cases = []
  fatalities = []

  first = 0
  firstDate = ''
  countryCurr = ''
  stateCurr = ''

  for i in range(train_data.shape[0]):
    useThis = train_data.iloc[i]["Province_State"]
    if not isinstance(train_data.iloc[i]["Province_State"], str):
      useThis = 'NONE'
    if train_data.iloc[i]["Country_Region"] == countryCurr and useThis == stateCurr:
      cases.append(train_data.iloc[i]["ConfirmedCases"]+cases[i-first])
      fatalities.append(train_data.iloc[i]["Fatalities"]+fatalities[i-first])
      date = dt.datetime.strptime(train_data.iloc[i]["Date"],'%Y-%m-%d')
      delta = date - firstDate
      dateNums.append(delta.days)
    else:
      if countryCurr != '':
        data.append([countryCurr, stateCurr, dateNums, cases, fatalities, firstDate])
        dateNums = []
        cases = []
        fatalities = []

      countryCurr = train_data.iloc[i]["Country_Region"]
      stateCurr = train_data.iloc[i]["Province_State"]
      if not isinstance(stateCurr, str):
        stateCurr = 'NONE'

      cases.append(train_data.iloc[i]["ConfirmedCases"])
      fatalities.append(train_data.iloc[i]["Fatalities"])
      date = dt.datetime.strptime(train_data.iloc[i]["Date"],'%Y-%m-%d')
      firstDate = date
      dateNums.append(0)
      first = i+1
  data.append([countryCurr, stateCurr, dateNums, cases, fatalities, firstDate])
  return data

# data = getData()

In [6]:
def logArray(array):
  copy = array.copy()
  for i in range(len(copy)):
    if copy[i] != 0.0:
      copy[i] = np.log(copy[i])
  return copy

In [8]:
def regressions(d):
  regressions = []
  for i in d:
    country = i[0]
    state = i[1]
    dates = i[2]
    cases = i[3]
    fatalities = i[4]
    firstDate = i[5]
    slopeC, interceptC, _, _, _ = stats.linregress(dates, logArray(cases))
    slopeF, interceptF, _, _, _ = stats.linregress(dates,logArray(fatalities))
    regressions.append([country, state, firstDate, [slopeC, interceptC], [slopeF, interceptF]])
  return regressions

def predict(regressions, country, state, date):
  for i in regressions:
    useThis = state
    if not isinstance(state, str):
      useThis = 'NONE'
    if i[0] == country and i[1] == useThis:
      date = dt.datetime.strptime(date,'%Y-%m-%d')
      delta = date - i[2]
      slope = i[3][0]
      intercept = i[3][1]
      case = np.exp((slope*delta.days)+intercept)
      slope = i[4][0]
      intercept = i[4][1]
      fatal = np.exp((slope*delta.days)+intercept)
      return case, fatal
  return "Not Found", "Not Found"


# print(predictFatalities(reg,'Afghanistan', 'NONE', '2020-04-02'))


NameError: ignored

In [9]:
data = getData()
reg = regressions(data)
print(predict(reg,'Afghanistan', 'NONE', '2020-04-02'))

submitData = []

for i in range(test_data.shape[0]):
  forecastId = test_data.iloc[i]['ForecastId']
  state = test_data.iloc[i]['Province_State']
  country = test_data.iloc[i]['Country_Region']
  date = test_data.iloc[i]['Date']
  predC, predF = predict(reg, country, state, date)
  if predC == 'Not Found':
    print(country, state, date)
  li = (forecastId, predC, predF)
  submitData.append(li)

with open("submission.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["ForecastID", "ConfirmedCases", "Fatalities"])  # write header
    writer.writerows(submitData)

(849.6038523482152, 40.166729378508066)


KeyboardInterrupt: ignored

Score is 1.74930